In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
%cd drive/MyDrive/nlp_project

/content/drive/MyDrive/nlp_project


In [ ]:
!pip install arabic-stopwords
!pip install qalsadi
!pip install pyarabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
import spacy
import string
import qalsadi.lemmatizer
from nltk.stem.isri import ISRIStemmer
from pyarabic.araby import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
import torch
from torch import nn
from tqdm import tqdm
import pickle


from pre_processing_post import processPost
from feature_extraction import get_ngram_features, get_word_embedding_features, avg_word_vector

In [4]:
# needed functions
def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

# Read train data

In [5]:
train_data = pd.read_csv('./DataSet/train.csv',sep=',',header=0)
test_data = pd.read_csv('./DataSet/dev.csv',sep=',',header=0)
train_data.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,celebrity,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,info_news,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,info_news,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,celebrity,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,personal,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",info_news,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,info_news,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,personal,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,unrelated,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,info_news,1


In [6]:
#remove first row that has the header
train_data['category'] = train_data['category'].astype('category').cat.codes
train_data.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,1,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,2,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,2,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,1,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,4,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",2,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,2,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,4,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,9,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,2,1


## Over Sampling

In [7]:
# !pip install imblearn

In [8]:
# from collections import Counter
# from imblearn.over_sampling import RandomOverSampler
# train_data=train_data.drop('category',axis=1)
# y=train_data['stance']
# print(Counter(train_data['stance']))
# train_data=train_data.drop('stance',axis=1)
# # define oversampling strategy
# oversample = RandomOverSampler(random_state=3)
# # fit and apply the transform
# train_data["text"], train_data['stance'] = oversample.fit_resample(train_data, y)
# print(Counter(train_data['stance']))

# Pre-Processing the tweets

In [9]:
print(train_data["text"][9])
train_data["text"] = train_data['text'].apply(lambda x: processPost(x))
test_data['text'] = test_data['text'].apply(lambda x: processPost(x))
print(train_data["text"][9])

دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية لقاح لعد ما اتابع الاخبار هم بكل مجالاتهم متفوقين وراح يطلع اللقاح قريباً؟<LF>#دعبول_دومه_مسحول
دعبول حضر من نت طلب قائد دول إسلام قاح عد تابع اخبار مجال متفوق طلع قاح قريبا دعبول دوم مسحول


## Ara2Vec Embedding

In [10]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

c:\Users\Nada\anaconda3\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'ar_pipeline' (0.0.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [11]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        # preprocessed = processPost(text)
        return self.tokenizer(text)

In [12]:
# Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)

## create vocablary

In [13]:
train_data_tokenized = train_data['text'].apply(tokenize)
test_data_tokenized = test_data['text'].apply(tokenize)
#merge all the sentences in one list
vocab = [item for sublist in train_data_tokenized for item in sublist]
vocab = list(set(vocab))
vocab.append('<فراغ>')
vocab.insert(0, '<مجهول>')
word2index = {word: i for i, word in enumerate(vocab)}

# Feature Extraction

## 1. TD-IDF

In [47]:
ngramdata_features, word_vectorizer = get_ngram_features(train_data)
ngramdata_features.head()

# save the model to disk
filename = 'TFIDFVectorizer.sav'
pickle.dump(word_vectorizer, open(filename, 'wb'))

LinearSVC Classifier

In [15]:
clf = LinearSVC()
pipe_tfidf = make_pipeline(word_vectorizer, clf)
pipe_tfidf.fit(train_data['text'], train_data['stance'])
print_report(pipe_tfidf, test_data['text'], test_data['stance'])

# save the model to disk
filename = 'LinearSVC_tfidf.sav'
pickle.dump(pipe_tfidf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.48      0.14      0.22        70
           0       0.45      0.28      0.34       126
           1       0.85      0.96      0.90       804

    accuracy                           0.81      1000
   macro avg       0.59      0.46      0.49      1000
weighted avg       0.78      0.81      0.78      1000

accuracy: 0.814


RandomForestClassifier

In [16]:
X_train_tfidf = word_vectorizer.fit_transform(train_data['text'])
X_test_tfidf = word_vectorizer.transform(test_data['text'])
rf = RandomForestClassifier()
rf_tfidf = rf.fit(X_train_tfidf, train_data['stance'])
y_pred = rf_tfidf.predict(X_test_tfidf)

print_report(rf_tfidf, X_test_tfidf, test_data['stance'])

# save the model to disk
filename = 'RandomForest_tfidf.sav'
pickle.dump(rf_tfidf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.52      0.17      0.26        70
           0       0.44      0.23      0.30       126
           1       0.84      0.95      0.89       804

    accuracy                           0.81      1000
   macro avg       0.60      0.45      0.48      1000
weighted avg       0.77      0.81      0.77      1000

accuracy: 0.807


naive bayes

In [46]:
X_train_tfidf = word_vectorizer.fit_transform(train_data['text'])
X_test_tfidf = word_vectorizer.transform(test_data['text'])
CNB_clr = ComplementNB()
CNB_clr_tfidf = CNB_clr.fit(X_train_tfidf, train_data['stance'])
print_report(CNB_clr_tfidf, X_test_tfidf, test_data['stance'])

# save the model to disk
filename = 'NaiveBayes_tfidf.sav'
pickle.dump(CNB_clr, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.30      0.44      0.36        70
           0       0.30      0.63      0.41       126
           1       0.94      0.74      0.83       804

    accuracy                           0.71      1000
   macro avg       0.52      0.61      0.53      1000
weighted avg       0.82      0.71      0.74      1000

accuracy: 0.707


## 2.CBOW

In [18]:
X_train_vect_avg, X_test_vect_avg = get_word_embedding_features(train_data, test_data)

e:\CMP\seventh term\Natural Language Processing\Arabic-Tweets-Sentiment-Classification\feature_extraction.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
e:\CMP\seventh term\Natural Language Processing\Arabic-Tweets-Sentiment-Classification\feature_extraction.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])


LinearSVC Classifier

In [49]:
clf = LinearSVC(class_weight={-1:0.66, 0:0.29, 1:0.05})
clf.fit(X_train_vect_avg, train_data['stance'])
print_report(clf, X_test_vect_avg, test_data['stance'])

# save the model to disk
filename = 'LinearSVC_CBOW.sav'
pickle.dump(clf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.21      0.11      0.15        70
           0       0.27      0.17      0.21       126
           1       0.83      0.91      0.87       804

    accuracy                           0.76      1000
   macro avg       0.44      0.40      0.41      1000
weighted avg       0.72      0.76      0.74      1000

accuracy: 0.761


c:\Users\Nada\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RandomForestClassifier

In [20]:
rf = RandomForestClassifier(class_weight="balanced")
rf_vect = rf.fit(X_train_vect_avg, train_data['stance'].values.ravel())
print_report(rf_vect, X_test_vect_avg, test_data['stance'])

# save the model to disk
filename = 'RandomForest_CBOW.sav'
pickle.dump(rf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.40      0.06      0.10        70
           0       0.48      0.08      0.14       126
           1       0.82      0.99      0.89       804

    accuracy                           0.81      1000
   macro avg       0.56      0.37      0.38      1000
weighted avg       0.75      0.81      0.74      1000

accuracy: 0.806


## 3.Ara2Vec

In [21]:
train_data_embeddings = np.array([np.array([nlp(i).vector for i in ls if i in vocab]) for ls in train_data["text"]], dtype=object)
test_data_embeddings = np.array([np.array([nlp(i).vector for i in ls if i in vocab]) for ls in test_data["text"]], dtype=object)

In [22]:
X_train_vect_avg, X_test_vect_avg = avg_word_vector(train_data_embeddings, test_data_embeddings)

LinearSVC Classifier

In [23]:
clf = LinearSVC(class_weight={-1:0.66, 0:0.29, 1:0.05})
clf.fit(X_train_vect_avg, train_data['stance'])
print_report(clf, X_test_vect_avg, test_data['stance'])

# save the model to disk
filename = 'LinearSVC_Ara2Vec.sav'
pickle.dump(clf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.20      0.11      0.14        70
           0       0.26      0.17      0.20       126
           1       0.83      0.91      0.87       804

    accuracy                           0.76      1000
   macro avg       0.43      0.40      0.41      1000
weighted avg       0.71      0.76      0.73      1000

accuracy: 0.759


c:\Users\Nada\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RandomForestClassifier

In [24]:
rf = RandomForestClassifier(class_weight="balanced")
rf_vect = rf.fit(X_train_vect_avg, train_data['stance'].values.ravel())
print_report(rf_vect, X_test_vect_avg, test_data['stance'])

# save the model to disk
filename = 'RandomForest_Ara2Vec.sav'
pickle.dump(rf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.40      0.06      0.10        70
           0       0.41      0.07      0.12       126
           1       0.82      0.99      0.89       804

    accuracy                           0.81      1000
   macro avg       0.54      0.37      0.37      1000
weighted avg       0.74      0.81      0.74      1000

accuracy: 0.805


## create initial embedding matrix 

In [25]:
weights_train_matrix = []
for word in vocab:
  weights_train_matrix.append(nlp(word).vector)

weights_train_matrix = torch.from_numpy(np.array(weights_train_matrix))
weights_train_matrix.size()

torch.Size([12538, 100])

# ArabicDataset
The class that impelements the dataset for arabic tweets

In [26]:
class ArabicDataset(torch.utils.data.Dataset):

  def __init__(self, x, y, pad):
    """
    This is the constructor of the ArabicDataset
    Inputs:
    - x: a list of lists where each list contains the ids of the tokens
    - y: a list of lists where each list contains the label of each token in the sentence
    - pad: the id of the <PAD> token (to be used for padding all sentences and labels to have the same length)
    """
    list_len = [len(i) for i in x]
    MAX_LENGTH = max(list_len) 
    for i in range(len(x)):
      x[i] = np.pad(x[i], (0, MAX_LENGTH-len(x[i])), 'constant', constant_values=(pad))

    self.x = torch.from_numpy(np.array(x)) 
    self.y = torch.from_numpy(np.array(y))

  def __len__(self):
    """
    This function should return the length of the dataset (the number of sentences)
    """
    return self.x.shape[0]

  def __getitem__(self, idx):
    """
    This function returns a subset of the whole dataset
    """
    return (self.x[idx], self.y[idx])

In [27]:
def create_emb_layer(weights_train_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_train_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_train_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

class Classifier(nn.Module):
  def __init__(self, vocab_size=len(vocab), embedding_dim=100, hidden_size=100, n_classes=3, n_layer=1):
    """
    The constructor of our NER model
    Inputs:
    - vacab_size: the number of unique words
    - embedding_dim: the embedding dimension
    - n_classes: the number of final classes (tags)
    """
    self.hidden_size = hidden_size
    super(Classifier, self).__init__()
    
    self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_train_matrix, True)
    self.hidden_size = hidden_size

    self.GRU = nn.GRU(input_size=embedding_dim, hidden_size=hidden_size, batch_first=True, num_layers=n_layer)

    self.linear = nn.Linear(hidden_size, n_classes)

  def forward(self, sentences):
    """
    This function does the forward pass of our model
    Inputs:
    - sentences: tensor of shape (batch_size, max_length)

    Returns:
    - final_output: tensor of shape (batch_size, max_length, n_classes)
    """

    final_output = None
    final_output, _ = self.GRU(self.embedding(sentences))
    final_output = final_output[:, -1, :]
    final_output = self.linear(final_output)
    return final_output

In [37]:
model = Classifier()
print(model)

Classifier(
  (embedding): Embedding(12538, 100)
  (GRU): GRU(100, 100, batch_first=True)
  (linear): Linear(in_features=100, out_features=3, bias=True)
)


# Training

In [38]:
def train(model, train_dataset, batch_size=32, epochs=10, learning_rate=0.001):
  """
  This function implements the training logic
  Inputs:
  - model: the model ot be trained
  - train_dataset: the training set of type NERDataset
  - batch_size: integer represents the number of examples per step
  - epochs: integer represents the total number of epochs (full training pass)
  - learning_rate: the learning rate to be used by the optimizer
  """

  # (1) create the dataloader of the training set (make the shuffle=True)
  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

  # (2) make the criterion cross entropy loss
  criterion = nn.CrossEntropyLoss(weight=torch.tensor([.5, .4, .1])) 

  # (3) create the optimizer (Adam)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # GPU configuration
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
    model = model.cuda()
    criterion = criterion.cuda()

  for epoch_num in range(epochs):
    if epoch_num % 5 == 0:
      learning_rate /= 10
    total_acc_train = 0
    total_loss_train = 0

    for train_input, train_label in tqdm(train_dataloader):

      # (4) move the train input to the device
      train_label = train_label.to(device)

      # (5) move the train label to the device
      train_input = train_input.to(device)

      # (6) do the forward pass
      output = model.forward(sentences=train_input)
      
      # (7) loss calculation (you need to think in this part how to calculate the loss correctly)
      batch_loss = criterion(output, train_label) 

      # (8) append the batch loss to the total_loss_train
      total_loss_train += batch_loss.item()
      
      # (9) calculate the batch accuracy (just add the number of correct predictions)
      argmax = torch.argmax(output, dim=1)
      acc = torch.sum(torch.eq(argmax, train_label))
      total_acc_train += acc

      # (10) zero your gradients
      optimizer.zero_grad()

      # (11) do the backward pass
      batch_loss.backward()

      # (12) update the weights with your optimizer
      optimizer.step()

    num_of_batches = len(train_dataset) / batch_size
    num_of_batches = int(num_of_batches) + 1
    # epoch loss
    epoch_loss = total_loss_train / num_of_batches
    
    # (13) calculate the accuracy
    epoch_acc = total_acc_train / len(train_dataset)

    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
        | Train Accuracy: {epoch_acc}\n')


In [39]:
train_data_tokenized_as_num = train_data_tokenized.apply(lambda x: [word2index[word] for word in x])
# apply the same tokenization to the test set
test_data_tokenized_as_num = test_data_tokenized.apply(lambda x: [word2index[word] for word in x if word in word2index])
train_dataset = ArabicDataset(list(train_data_tokenized_as_num), train_data['stance'] + 1, word2index['<فراغ>'])
test_dataset = ArabicDataset(list(test_data_tokenized_as_num), test_data['stance'] + 1, word2index['<فراغ>'])

In [40]:
train(model, train_dataset, epochs=10, learning_rate=0.001)

100%|██████████| 219/219 [00:18<00:00, 12.00it/s]


Epochs: 1 | Train Loss: 1.0329111667528545         | Train Accuracy: 0.7449914216995239



100%|██████████| 219/219 [00:19<00:00, 11.03it/s]


Epochs: 2 | Train Loss: 1.0293194286899479         | Train Accuracy: 0.7925014495849609



100%|██████████| 219/219 [00:19<00:00, 11.52it/s]


...

Epochs: 8 | Train Loss: 0.5115706066698789         | Train Accuracy: 0.8148254156112671



100%|██████████| 219/219 [00:19<00:00, 11.31it/s]


Epochs: 9 | Train Loss: 0.42234649260838825         | Train Accuracy: 0.8457355499267578



100%|██████████| 219/219 [00:19<00:00, 11.34it/s]

Epochs: 10 | Train Loss: 0.3685627749793606         | Train Accuracy: 0.8567544221878052



In [41]:
# save the model to disk
filename = './models/GRU_Ara2Vec.sav'
torch.save(model.state_dict(), filename)

# Evaluation

In [42]:
def evaluate(model, test_dataset, batch_size=32):
  """
  This function takes a model and evaluates its performance (accuracy) on a test data
  Inputs:
  - model: the model
  - test_dataset: dataset of type ArabicDataset
  """
  
  # (1) create the test data loader
  test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  # GPU Configuration
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
    model = model.cuda()

  total_acc_test = 0.0
  
  y_test = [] 
  y_predected = [] 
  # (2) disable gradients
  with torch.no_grad():
    report = None
    for test_input, test_label in tqdm(test_dataloader):
      # (3) move the test input to the device
      test_label = test_label.to(device)

      # (4) move the test label to the device
      test_input = test_input.to(device)

      # (5) do the forward pass
      output = model.forward(sentences=test_input)

      # accuracy calculation (just add the correct predicted items to total_acc_test)
      acc = torch.sum(torch.eq(torch.argmax(output, dim=1), test_label))
      total_acc_test += acc
      
      # f1 score calculation
      y_test +=(list(test_label.view(-1)))
      y_predected +=(list(torch.argmax(output, dim=1).view(-1)))

    # (6) calculate the over all accuracy
    total_acc_test /= len(test_dataset)

  report = metrics.classification_report(y_test, y_predected)
  print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_predected)))
  print(report)
  
  print(f'\nTest Accuracy: {total_acc_test}')

In [43]:
evaluate(model, test_dataset)

100%|██████████| 32/32 [00:00<00:00, 43.65it/s]


accuracy: 0.721
              precision    recall  f1-score   support

           0       0.24      0.34      0.29        70
           1       0.29      0.51      0.37       126
           2       0.93      0.79      0.85       804

    accuracy                           0.72      1000
   macro avg       0.49      0.55      0.50      1000
weighted avg       0.80      0.72      0.75      1000


Test Accuracy: 0.7210000157356262
